In [ ]:
# 전력 수요 예측 프로젝트
# 분석 관점: 1) 시간 단위, 2) 계절 단위(3개월), 3) 연도 단위 예측
# 조원: 김도영, 김남훈

---

## 📁 프로젝트 개요 및 분석 관점 설명 (출처: 프로젝트.pdf 1~2쪽)
- 주제: 전력 수요 데이터를 기반으로 시계열 예측을 수행하여 향후 수요 변화 양상을 분석함
- 분석 관점:
    - 시간 단위: 실시간 대응과 단기 피크 수요 예측
    - 계절 단위: 분기별 전력 소비 변화 분석 (봄/여름/가을/겨울)
    - 연도 단위: 장기 추세 예측을 통한 에너지 정책 수립 기반 마련
- 평가 항목 기반 구성: 모델 설명, 시각화, 성능 비교(MAE, RMSE), 해석 포함

---

# -- 1. 라이브러리 불러오기 및 기본 설정 --
# (출처: 프로젝트.pdf 2쪽 - 환경 구성과 시각화 기준 설명 기반)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SARIMAX, seasonal_decompose

# 한글 폰트 설정 (Colab 환경)
import platform
if platform.system() == 'Linux':
    !apt-get -qq install fonts-nanum > /dev/null
    plt.rc('font', family='NanumGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# -- 2. 데이터 불러오기 및 전처리 --
# (출처: 프로젝트.pdf 3쪽 - 시간 단위 처리 방식 설명 기반)
df = pd.read_csv("한국전력거래소_시간별 전국 전력수요량_20241231.csv", encoding="cp949")
df_long = df.melt(id_vars='날짜', var_name='시간', value_name='수요량(MWh)')
df_long['시간_num'] = df_long['시간'].str.replace('시', '').astype(int)
df_long['일시'] = pd.to_datetime(df_long['날짜']) + pd.to_timedelta(df_long['시간_num'] % 24, unit='h')
df_long.loc[df_long['시간_num'] == 24, '일시'] += pd.Timedelta(days=1)
df_long = df_long[['일시', '수요량(MWh)']].sort_values('일시')
df_long = df_long.set_index('일시')
